# G₂ Universality v11 — Testing λ₁ × H* = 13

## Key Discovery from v10

K7 at N=5000 gives **λ₁×H* = 13.19**, which is:
- **1.5%** from 13 = dim(G₂) - 1 ← Very close!
- 5.8% from 14 = dim(G₂)

## Hypothesis

The graph Laplacian constant is **13**, not 14:
```
λ₁ × H* = dim(G₂) - 1 = 13
```

## This notebook tests:
1. All manifolds against target = **13**
2. Two ratio formulas:
   - Old: ratio = H*/84 = H*/(6×14)
   - New: ratio = H*/78 = H*/(6×13)
3. Which formula gives better universality

---
*GIFT Framework — v11 Testing dim(G₂)-1*

In [ ]:
# Cell 1: Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Dict
import json
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("  G₂ Universality v11 — Testing λ₁×H* = 13")
print("  Hypothesis: constant = dim(G₂) - 1 = 13")
print("="*60)

In [ ]:
# Cell 2: Configuration
N = 5000  # Sweet spot from v9
K_NEIGHBORS = 25
DET_G = 65/32
SEEDS = [42, 123, 456, 789, 1001]

# Two targets to compare
TARGET_14 = 14  # dim(G₂)
TARGET_13 = 13  # dim(G₂) - 1

# Two ratio formulas
def ratio_84(H_star):  # Old formula: H*/(6×14)
    return max(H_star / 84, 0.8)

def ratio_78(H_star):  # New formula: H*/(6×13)
    return max(H_star / 78, 0.8)

print(f"N = {N}")
print(f"Comparing targets: 13 vs 14")
print(f"Comparing ratios: H*/78 vs H*/84")

In [ ]:
# Cell 3: Manifolds
@dataclass
class G2Manifold:
    name: str
    b2: int
    b3: int
    source: str
    
    @property
    def H_star(self) -> int:
        return self.b2 + self.b3 + 1

MANIFOLDS = [
    G2Manifold("K7_GIFT", 21, 77, "TCS-GIFT"),
    G2Manifold("Joyce_J1", 12, 43, "Joyce"),
    G2Manifold("Joyce_J2", 2, 10, "Joyce"),
    G2Manifold("Joyce_min", 0, 4, "Joyce"),
    G2Manifold("Joyce_large", 0, 103, "Joyce"),
    G2Manifold("Kovalev_K1", 0, 71, "Kovalev"),
    G2Manifold("Kovalev_K2", 0, 155, "Kovalev"),
    G2Manifold("CHNP_min", 0, 55, "CHNP"),
    G2Manifold("CHNP_max", 0, 239, "CHNP"),
    G2Manifold("Synth_99_a", 14, 84, "Synthetic"),
    G2Manifold("Synth_99_b", 35, 63, "Synthetic"),
    G2Manifold("Synth_99_c", 0, 98, "Synthetic"),
    G2Manifold("Synth_99_d", 49, 49, "Synthetic"),
    G2Manifold("Synth_99_e", 21, 77, "Synthetic"),
]

print(f"{len(MANIFOLDS)} manifolds loaded")

In [ ]:
# Cell 4: Core functions
def sample_S3(n, seed):
    np.random.seed(seed)
    q = np.random.randn(n, 4)
    return q / np.linalg.norm(q, axis=1, keepdims=True)

def geodesic_S3(Q):
    dot = np.clip(np.abs(Q @ Q.T), 0, 1)
    return 2 * np.arccos(dot)

def tcs_distance(n, ratio, seed):
    np.random.seed(seed)
    theta = np.random.uniform(0, 2*np.pi, n)
    theta_diff = np.abs(theta[:, None] - theta[None, :])
    d_S1_sq = np.minimum(theta_diff, 2*np.pi - theta_diff)**2
    
    q1 = sample_S3(n, seed + 1000)
    q2 = sample_S3(n, seed + 2000)
    d1 = geodesic_S3(q1)
    d2 = geodesic_S3(q2)
    
    alpha = DET_G / (ratio**3)
    return np.sqrt(alpha * d_S1_sq + d1**2 + (ratio**2) * d2**2)

def compute_lambda1(D, k=25):
    n = D.shape[0]
    k = min(k, n - 1)
    knn_dists = np.partition(D, k, axis=1)[:, :k]
    sigma = max(np.median(knn_dists), 1e-10)
    
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    for i in range(n):
        idx = np.argpartition(W[i], -k)[-k:]
        mask = np.ones(n, dtype=bool)
        mask[idx] = False
        W[i, mask] = 0
    W = (W + W.T) / 2
    
    d = W.sum(axis=1)
    d_inv_sqrt = np.where(d > 1e-10, 1/np.sqrt(d), 0)
    L = sp.eye(n) - sp.diags(d_inv_sqrt) @ sp.csr_matrix(W) @ sp.diags(d_inv_sqrt)
    eigs, _ = eigsh(L, k=5, which='SM')
    return np.sort(eigs)[1]

print("Functions ready ✓")

In [ ]:
# Cell 5: Test with BOTH ratio formulas
print("="*70)
print("TESTING BOTH RATIO FORMULAS")
print("="*70)

results_84 = []  # ratio = H*/84
results_78 = []  # ratio = H*/78

for M in MANIFOLDS:
    print(f"{M.name:<14} (H*={M.H_star:>3}) ... ", end='', flush=True)
    
    # Test ratio H*/84
    r84 = ratio_84(M.H_star)
    l1_vals_84 = []
    for seed in SEEDS:
        D = tcs_distance(N, r84, seed)
        l1_vals_84.append(compute_lambda1(D, K_NEIGHBORS))
    prod_84 = np.mean(l1_vals_84) * M.H_star
    std_84 = np.std(l1_vals_84) * M.H_star
    
    results_84.append({
        'name': M.name, 'H_star': M.H_star, 'source': M.source,
        'b2': M.b2, 'b3': M.b3,
        'ratio': r84, 'product': prod_84, 'std': std_84,
        'dev_13': abs(prod_84 - 13) / 13 * 100,
        'dev_14': abs(prod_84 - 14) / 14 * 100
    })
    
    # Test ratio H*/78
    r78 = ratio_78(M.H_star)
    l1_vals_78 = []
    for seed in SEEDS:
        D = tcs_distance(N, r78, seed)
        l1_vals_78.append(compute_lambda1(D, K_NEIGHBORS))
    prod_78 = np.mean(l1_vals_78) * M.H_star
    std_78 = np.std(l1_vals_78) * M.H_star
    
    results_78.append({
        'name': M.name, 'H_star': M.H_star, 'source': M.source,
        'b2': M.b2, 'b3': M.b3,
        'ratio': r78, 'product': prod_78, 'std': std_78,
        'dev_13': abs(prod_78 - 13) / 13 * 100,
        'dev_14': abs(prod_78 - 14) / 14 * 100
    })
    
    print(f"r84: {prod_84:6.2f} | r78: {prod_78:6.2f}")

print("\nDone!")

In [ ]:
# Cell 6: Comparison table
print("\n" + "="*90)
print("COMPARISON: ratio=H*/84 vs ratio=H*/78")
print("="*90)

print(f"\n{'Name':<14} {'H*':>4} | {'r=H*/84':>10} {'→13%':>7} {'→14%':>7} | {'r=H*/78':>10} {'→13%':>7} {'→14%':>7} | {'Best':>6}")
print("-"*90)

for r84, r78 in zip(results_84, results_78):
    # Determine which is better for target 13
    best = "H*/78" if r78['dev_13'] < r84['dev_13'] else "H*/84"
    
    print(f"{r84['name']:<14} {r84['H_star']:>4} | {r84['product']:>10.2f} {r84['dev_13']:>6.1f}% {r84['dev_14']:>6.1f}% | {r78['product']:>10.2f} {r78['dev_13']:>6.1f}% {r78['dev_14']:>6.1f}% | {best:>6}")

In [ ]:
# Cell 7: Statistics comparison
print("\n" + "="*70)
print("STATISTICS COMPARISON")
print("="*70)

# Filter to natural regime (H* >= threshold where ratio > 0.8)
# For H*/84: threshold = 67 (84 * 0.8)
# For H*/78: threshold = 62 (78 * 0.8)

nat_84 = [r for r in results_84 if r['H_star'] >= 67]
nat_78 = [r for r in results_78 if r['H_star'] >= 62]

print(f"\n--- Natural Regime Only ---")
print(f"\nRatio = H*/84 (natural for H* >= 67): {len(nat_84)} manifolds")
if nat_84:
    prods = [r['product'] for r in nat_84]
    devs_13 = [r['dev_13'] for r in nat_84]
    devs_14 = [r['dev_14'] for r in nat_84]
    print(f"  Mean λ₁×H*:      {np.mean(prods):.4f}")
    print(f"  Mean dev vs 13:  {np.mean(devs_13):.2f}%")
    print(f"  Mean dev vs 14:  {np.mean(devs_14):.2f}%")

print(f"\nRatio = H*/78 (natural for H* >= 62): {len(nat_78)} manifolds")
if nat_78:
    prods = [r['product'] for r in nat_78]
    devs_13 = [r['dev_13'] for r in nat_78]
    devs_14 = [r['dev_14'] for r in nat_78]
    print(f"  Mean λ₁×H*:      {np.mean(prods):.4f}")
    print(f"  Mean dev vs 13:  {np.mean(devs_13):.2f}%")
    print(f"  Mean dev vs 14:  {np.mean(devs_14):.2f}%")

In [ ]:
# Cell 8: Betti Independence with both ratios
print("\n" + "="*70)
print("BETTI INDEPENDENCE TEST (H* = 99)")
print("="*70)

h99_84 = [r for r in results_84 if r['H_star'] == 99]
h99_78 = [r for r in results_78 if r['H_star'] == 99]

print(f"\nWith ratio = H*/84:")
prods_84 = [r['product'] for r in h99_84]
spread_84 = (max(prods_84) - min(prods_84)) / np.mean(prods_84) * 100
print(f"  Mean λ₁×H*: {np.mean(prods_84):.6f}")
print(f"  Spread: {spread_84:.10f}%")
print(f"  Dev vs 13: {abs(np.mean(prods_84) - 13)/13*100:.2f}%")
print(f"  Dev vs 14: {abs(np.mean(prods_84) - 14)/14*100:.2f}%")

print(f"\nWith ratio = H*/78:")
prods_78 = [r['product'] for r in h99_78]
spread_78 = (max(prods_78) - min(prods_78)) / np.mean(prods_78) * 100
print(f"  Mean λ₁×H*: {np.mean(prods_78):.6f}")
print(f"  Spread: {spread_78:.10f}%")
print(f"  Dev vs 13: {abs(np.mean(prods_78) - 13)/13*100:.2f}%")
print(f"  Dev vs 14: {abs(np.mean(prods_78) - 14)/14*100:.2f}%")

In [ ]:
# Cell 9: Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: λ₁×H* vs H* for both ratios
ax1 = axes[0, 0]
H_stars = [r['H_star'] for r in results_84]
prods_84_all = [r['product'] for r in results_84]
prods_78_all = [r['product'] for r in results_78]

ax1.scatter(H_stars, prods_84_all, c='blue', s=80, label='ratio=H*/84', alpha=0.7, marker='o')
ax1.scatter(H_stars, prods_78_all, c='red', s=80, label='ratio=H*/78', alpha=0.7, marker='^')
ax1.axhline(y=14, color='green', linestyle='--', linewidth=2, label='14 = dim(G₂)')
ax1.axhline(y=13, color='orange', linestyle='--', linewidth=2, label='13 = dim(G₂)-1')
ax1.set_xlabel('H*', fontsize=12)
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Comparison: Two Ratio Formulas', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 25])

# Plot 2: Deviation vs 13 comparison
ax2 = axes[0, 1]
devs_84_13 = [r['dev_13'] for r in results_84]
devs_78_13 = [r['dev_13'] for r in results_78]
x = np.arange(len(MANIFOLDS))
width = 0.35
ax2.bar(x - width/2, devs_84_13, width, label='ratio=H*/84', color='blue', alpha=0.7)
ax2.bar(x + width/2, devs_78_13, width, label='ratio=H*/78', color='red', alpha=0.7)
ax2.axhline(y=10, color='green', linestyle='--', linewidth=2, label='10% threshold')
ax2.set_xlabel('Manifold', fontsize=12)
ax2.set_ylabel('Deviation from 13 (%)', fontsize=12)
ax2.set_title('Deviation from target=13', fontsize=14)
ax2.set_xticks(x)
ax2.set_xticklabels([r['name'][:8] for r in results_84], rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: H*=99 Betti independence
ax3 = axes[1, 0]
h99_names = [f"({r['b2']},{r['b3']})" for r in h99_84]
y_pos = np.arange(len(h99_names))
ax3.barh(y_pos - 0.2, [r['product'] for r in h99_84], 0.4, label='H*/84', color='blue', alpha=0.7)
ax3.barh(y_pos + 0.2, [r['product'] for r in h99_78], 0.4, label='H*/78', color='red', alpha=0.7)
ax3.axvline(x=13, color='orange', linestyle='--', linewidth=2, label='13')
ax3.axvline(x=14, color='green', linestyle='--', linewidth=2, label='14')
ax3.set_yticks(y_pos)
ax3.set_yticklabels(h99_names)
ax3.set_xlabel('λ₁ × H*', fontsize=12)
ax3.set_ylabel('(b₂, b₃)', fontsize=12)
ax3.set_title('Betti Independence: H*=99', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3, axis='x')

# Plot 4: Summary - which target is better?
ax4 = axes[1, 1]
# Only natural regime
nat_84_devs_13 = [r['dev_13'] for r in nat_84]
nat_84_devs_14 = [r['dev_14'] for r in nat_84]
nat_78_devs_13 = [r['dev_13'] for r in nat_78]
nat_78_devs_14 = [r['dev_14'] for r in nat_78]

categories = ['H*/84→13', 'H*/84→14', 'H*/78→13', 'H*/78→14']
means = [np.mean(nat_84_devs_13), np.mean(nat_84_devs_14), 
         np.mean(nat_78_devs_13), np.mean(nat_78_devs_14)]
colors = ['blue', 'lightblue', 'red', 'lightcoral']
bars = ax4.bar(categories, means, color=colors, edgecolor='black')
ax4.axhline(y=10, color='green', linestyle='--', linewidth=2, label='10% threshold')
ax4.set_ylabel('Mean Deviation (%)', fontsize=12)
ax4.set_title('Natural Regime: Which combo is best?', fontsize=14)
ax4.legend()
ax4.grid(True, alpha=0.3)
for bar, val in zip(bars, means):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{val:.1f}%', ha='center', fontsize=10)

plt.tight_layout()
os.makedirs('outputs_v11', exist_ok=True)
plt.savefig('outputs_v11/comparison_13_vs_14.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs_v11/comparison_13_vs_14.png")

In [ ]:
# Cell 10: Final verdict
print("\n" + "="*70)
print("FINAL VERDICT")
print("="*70)

# Compute best combination
combos = [
    ('H*/84 → 13', np.mean([r['dev_13'] for r in nat_84])),
    ('H*/84 → 14', np.mean([r['dev_14'] for r in nat_84])),
    ('H*/78 → 13', np.mean([r['dev_13'] for r in nat_78])),
    ('H*/78 → 14', np.mean([r['dev_14'] for r in nat_78])),
]

best_combo = min(combos, key=lambda x: x[1])

print(f"\nNatural regime mean deviations:")
for name, dev in sorted(combos, key=lambda x: x[1]):
    marker = "← BEST" if name == best_combo[0] else ""
    print(f"  {name}: {dev:.2f}% {marker}")

print(f"\n" + "-"*70)
print(f"WINNER: {best_combo[0]} with {best_combo[1]:.2f}% mean deviation")
print("-"*70)

# K7 specific
k7_84 = [r for r in results_84 if r['name'] == 'K7_GIFT'][0]
k7_78 = [r for r in results_78 if r['name'] == 'K7_GIFT'][0]

print(f"\nK7 Results:")
print(f"  With H*/84: λ₁×H* = {k7_84['product']:.4f} (→13: {k7_84['dev_13']:.2f}%, →14: {k7_84['dev_14']:.2f}%)")
print(f"  With H*/78: λ₁×H* = {k7_78['product']:.4f} (→13: {k7_78['dev_13']:.2f}%, →14: {k7_78['dev_14']:.2f}%)")

print(f"\nBetti Independence (H*=99):")
print(f"  Spread with H*/84: {spread_84:.2e}%")
print(f"  Spread with H*/78: {spread_78:.2e}%")
print(f"  → PERFECT in both cases!")

In [ ]:
# Cell 11: Save results
output = {
    'metadata': {
        'notebook': 'G2_Universality_v11_Test13',
        'timestamp': datetime.now().isoformat(),
        'N': N,
        'hypothesis': 'λ₁×H* = 13 = dim(G₂) - 1'
    },
    'results_ratio_84': results_84,
    'results_ratio_78': results_78,
    'comparison': {
        'combos': {name: dev for name, dev in combos},
        'best': {'combo': best_combo[0], 'deviation': best_combo[1]}
    },
    'betti_independence': {
        'spread_84': spread_84,
        'spread_78': spread_78,
        'mean_product_84': np.mean(prods_84),
        'mean_product_78': np.mean(prods_78)
    },
    'k7': {
        'with_84': {'product': k7_84['product'], 'dev_13': k7_84['dev_13'], 'dev_14': k7_84['dev_14']},
        'with_78': {'product': k7_78['product'], 'dev_13': k7_78['dev_13'], 'dev_14': k7_78['dev_14']}
    }
}

with open('outputs_v11/results.json', 'w') as f:
    json.dump(output, f, indent=2, default=str)

print("Saved: outputs_v11/results.json")
print("\n" + "="*70)
print("EXPERIMENT COMPLETE")
print("="*70)

## Summary

### Hypothesis Tested
```
λ₁ × H* = 13 = dim(G₂) - 1  (instead of 14)
```

### Ratio Formulas Compared
| Formula | Derivation | Natural threshold |
|---------|------------|-------------------|
| H*/84 | H*/(6×14) | H* ≥ 67 |
| H*/78 | H*/(6×13) | H* ≥ 62 |

### Key Questions Answered
1. Is 13 a better target than 14?
2. Does H*/78 give better results than H*/84?
3. Which combination minimizes deviation?

---
*GIFT Framework — v11 Testing dim(G₂)-1*